In [83]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import ssl
import os
import re

In [84]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [85]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    #If there are no upvotes replace '' with 0
    if first_num == '':
        first_num = 0
    return first_num
    

In [86]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [87]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [88]:
#get list of urls
url_list = list(df['url'].values)

In [89]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [90]:
def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    page = requests.get(url)
   
    #create soup object with htmal parser
    soup = BeautifulSoup(page.content, 'html.parser')

    answers = soup.find_all('script', {"type": "application/ld+json"})[0]
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "upvoteCount", "answerCount", "followerCount", "name"]
    #columns = ["Answer", "upvoteCount"]
    x_shape = (len(answers_list), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)
    columns = columns[1:]

    for i, ans in enumerate(answers_list):    
        #get list of categories that exist in answer post
        existing_columns = [col for col in columns if col in ans]
        missing_columns = [col for col in columns if col not in ans]

        #fill all missing features in post with UNKNOWN
        answers_df.loc[i, missing_columns] = 'UNKNOWN'         
        
        n = len(existing_columns)
        c = iter(existing_columns)
        curr_c = next(c)

        for j in range(n):          
            #Fill first element column which is 'Answer'   
            try:
                next_c = next(c)
            except StopIteration:
                pass
                        
            if j==0:
                split_post = ans.split(curr_c)
                ans_text = split_post[0]
                answers_df['Answer'].iloc[i] = ans_text
                split_post = split_post[1] 
               
                
            #numeric categories have Count in their names        
            if "Count" in curr_c:
                #in case the category is
                if curr_c!=existing_columns[-1]:
                    #get text relevant to c -column
                    split_post = split_post.split(curr_c)
                    inter_split = split_post[-1].split(next_c)
                    #insert regex helper function here----!
                    #if category is numeric get numbe
                    answers_df[curr_c].iloc[i] = str_first_num(inter_split[0])
                    split_post = inter_split[-1]
                    curr_c = next_c
                    continue 
                else:
                #    split_post = split_post.split(curr_c)
                    answers_df[curr_c].iloc[i] = str_first_num(split_post)
                    continue
        
            #insert regex helper function here----!
            answers_df[curr_c].iloc[i] = split_post 
            curr_c = next_c 
    return answers_df 

In [91]:
columns = ["question", "Answer", "upvoteCount", "answerCount", "followerCount", "name"]
df = pd.DataFrame(data = [], columns = columns)

for i, url in enumerate(url_list):
    temp_df = answers_into_df(url)
    temp_df["question"] = url
    #rearrange columns order to match main df columns order
    temp_df = temp_df[columns]

    df = df.append(temp_df)


C:\Users\Ravit\AppData\Local\Temp\ipykernel_15844\705147078.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df['Answer'].iloc[i] = ans_text
C:\Users\Ravit\AppData\Local\Temp\ipykernel_15844\705147078.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df[curr_c].iloc[i] = str_first_num(split_post)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_15844\1010041338.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_15844\705147078.py:62: SettingWith

In [92]:
df

,question,Answer,upvoteCount,answerCount,followerCount,name
0,https://www.quora.com/What-will-the-worlds-tec...,"""50 years is a very long time. The way I make...",348,UNKNOWN,UNKNOWN,UNKNOWN
1,https://www.quora.com/What-will-the-worlds-tec...,"""Faster elevators capable of moving sideways....",3970,12202,8903,""": ""Richard Kenneth Eng"", ""url"": ""https://www...."
2,https://www.quora.com/What-will-the-worlds-tec...,"""Well, lucky for you, I just returned from 50...",4,1,0,""": ""Phillip Jensen"", ""url"": ""https://www.quora..."
3,https://www.quora.com/What-will-the-worlds-tec...,"""the only answer I can give you is that in th...",3,3871,970,""": ""Ernie Dunbar"", ""url"": ""https://www.quora.c..."
4,https://www.quora.com/What-will-the-worlds-tec...,"""I imagine I will be completely wrong.\n\nIf ...",4,983,103,""": ""Steve Schaefer"", ""url"": ""https://www.quora..."
...,...,...,...,...,...,...
5,https://www.quora.com/What-are-the-solutions-t...,"""There are 3 approaches.\n\nOption 1 : Myanma...",16,13273,1453,""": ""Kathleen Listman"", ""url"": ""https://www.quo..."
6,https://www.quora.com/What-are-the-solutions-t...,"""Technology is expensive and it does not impr...",4,UNKNOWN,UNKNOWN,UNKNOWN
7,https://www.quora.com/What-are-the-solutions-t...,"""Here are some ways I use technology in my cl...",55,1400,120,""": ""Dave Wheatley"", ""url"": ""https://www.quora...."
8,https://www.quora.com/What-are-the-solutions-t...,"""Read Peter Diamandis\u2019 book \u2018Abunda...",3,UNKNOWN,UNKNOWN,UNKNOWN


In [93]:
#save df to csv
save_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info"
final_path = os.path.join(save_path, "ET_RCS_22SEP15_quora_data.csv")
df.to_csv(final_path)